### Download Data

In [1]:
! wget -q https://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy

### Import Dependencies

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from Seq2Seq import Seq2Seq
from torch.utils.data import DataLoader

import io
import imageio
from ipywidgets import widgets, HBox

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Import Data and Create Dataloaders

In [2]:
# Load Data as Numpy Array
MovingMNIST = np.load('mnist_test_seq.npy').transpose(1, 0, 2, 3)

# Shuffle Data
np.random.shuffle(MovingMNIST)

# Train, Test, Validation splits
train_data = MovingMNIST[:8000]         
val_data = MovingMNIST[8000:9000]       
test_data = MovingMNIST[9000:10000]     

def collate(batch):

    # Add channel dim, scale pixels between 0 and 1, send to GPU
    batch = torch.tensor(batch).unsqueeze(1)     
    batch = batch / 255.0                        
    batch = batch.to(device)                     

    # Randomly pick 10 frames as input, 11th frame is target
    rand = np.random.randint(10,20)                     
    return batch[:,:,rand-10:rand], batch[:,:,rand]     


# Training Data Loader
train_loader = DataLoader(train_data, shuffle=True, 
                        batch_size=16, collate_fn=collate)

# Validation Data Loader
val_loader = DataLoader(val_data, shuffle=True, 
                        batch_size=16, collate_fn=collate)

### Visualize Data

In [3]:
# Get a batch
input, _ = next(iter(val_loader))

# Reverse process before displaying
input = input.cpu().numpy() * 255.0     

for video in input.squeeze(1)[:3]:          # Loop over videos
    with io.BytesIO() as gif:
        imageio.mimsave(gif,video.astype(np.uint8),"GIF",fps=5)
        display(HBox([widgets.Image(value=gif.getvalue())]))

### Instantiate Model, Optimizer and Loss

In [4]:
# The input video frames are grayscale, thus single channel
model = Seq2Seq(num_channels=1, num_kernels=64, 
kernel_size=(3, 3), padding=(1, 1), activation="relu", 
frame_size=(64, 64), num_layers=3).to(device)

#optim = Adam(model.parameters(), lr=1e-4)

# NOTE: only pass the parameters where p.requires_grad == True to the optimizer! Important!
optim = torch.optim.Adam(
    [p for p in model.parameters() if p.requires_grad],
    lr=1e-4,
)

for n, p in model.named_parameters():
    if p.requires_grad:
        print(f"Paramter {n} requires grad")

# Binary Cross Entropy, target pixel values either 0 or 1
criterion = nn.BCELoss(reduction='sum')

Paramter sequential.convlstm1.convLSTMcell.conv.scores requires grad
Paramter sequential.convlstm2.convLSTMcell.conv.scores requires grad
Paramter sequential.convlstm3.convLSTMcell.conv.scores requires grad
Paramter conv.scores requires grad


### Train for about 20 epochs

In [5]:
num_epochs = 20

for epoch in range(1, num_epochs+1):
    
    train_loss = 0                                                 
    model.train()                                                  
    for batch_num, (input, target) in enumerate(train_loader, 1):  
        output = model(input)                                     
        loss = criterion(output.flatten(), target.flatten())       
        loss.backward()                                            
        optim.step()                                               
        optim.zero_grad()                                           
        train_loss += loss.item()   
    train_loss /= len(train_loader.dataset)                       

    val_loss = 0                                                 
    model.eval()                                                   
    with torch.no_grad():                                          
        for input, target in val_loader:                          
            output = model(input)                                   
            loss = criterion(output.flatten(), target.flatten())   
            val_loss += loss.item()                                
    val_loss /= len(val_loader.dataset)                            

    print("Epoch:{} Training Loss:{:.2f} Validation Loss:{:.2f}\n".format(
        epoch, train_loss, val_loss))

Epoch:1 Training Loss:385.55 Validation Loss:312.14

Epoch:2 Training Loss:307.25 Validation Loss:300.11

Epoch:3 Training Loss:297.69 Validation Loss:295.52

Epoch:4 Training Loss:291.87 Validation Loss:287.06

Epoch:5 Training Loss:287.32 Validation Loss:288.96

Epoch:6 Training Loss:283.68 Validation Loss:281.07

Epoch:7 Training Loss:282.14 Validation Loss:276.75

Epoch:8 Training Loss:279.48 Validation Loss:278.42

Epoch:9 Training Loss:277.45 Validation Loss:276.69

Epoch:10 Training Loss:274.81 Validation Loss:272.26

Epoch:11 Training Loss:272.85 Validation Loss:267.16

Epoch:12 Training Loss:270.41 Validation Loss:267.17

Epoch:13 Training Loss:270.20 Validation Loss:269.95

Epoch:14 Training Loss:267.67 Validation Loss:271.15

Epoch:15 Training Loss:267.27 Validation Loss:266.75

Epoch:16 Training Loss:268.47 Validation Loss:264.39

Epoch:17 Training Loss:265.33 Validation Loss:266.67

Epoch:18 Training Loss:264.65 Validation Loss:267.31

Epoch:19 Training Loss:264.71 Validat

### Visualize what our model has learned so far

In [6]:
def collate_test(batch):

    # Last 10 frames are target
    target = np.array(batch)[:,10:]                     
    
    # Add channel dim, scale pixels between 0 and 1, send to GPU
    batch = torch.tensor(batch).unsqueeze(1)          
    batch = batch / 255.0                             
    batch = batch.to(device)                          
    return batch, target

# Test Data Loader
test_loader = DataLoader(test_data,shuffle=True, 
                         batch_size=3, collate_fn=collate_test)

# Get a batch
batch, target = next(iter(test_loader))

# Initialize output sequence
output = np.zeros(target.shape, dtype=np.uint8)

# Loop over timesteps
for timestep in range(target.shape[1]):
  input = batch[:,:,timestep:timestep+10]   
  output[:,timestep]=(model(input).squeeze(1).cpu()>0.5)*255.0

In [7]:
for tgt, out in zip(target, output):       # Loop over samples
    
    # Write target video as gif
    with io.BytesIO() as gif:
        imageio.mimsave(gif, tgt, "GIF", fps = 5)    
        target_gif = gif.getvalue()

    # Write output video as gif
    with io.BytesIO() as gif:
        imageio.mimsave(gif, out, "GIF", fps = 5)    
        output_gif = gif.getvalue()

    display(HBox([widgets.Image(value=target_gif), 
                  widgets.Image(value=output_gif)]))